***GENERATED CODE FOR delcols PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('hospital_pk')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run delcolsHooks.ipynb
try:
	#sourcePreExecutionHook()

	filterdatanewbkqi = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1718387357966_filter_data_new.csv', 'filename': 'filter_data_new.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/TimeSeries/CoronaHospCapa/filter_data_new.csv', 'viewFileName': 'filter_data_new.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run delcolsHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(filterdatanewbkqi,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "hospital_pk", "transformation_label": "novalue"}], "feature": "hospital_pk", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "7422", "mean": "434353.62", "stddev": "4100.59", "min": "431309", "max": "440031", "missing": "0", "distinct": "52"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hospital_pk"}, {"transformationsData": [{"feature_label": "collection_week", "transformation_label": "novalue"}], "feature": "collection_week", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7422", "mean": "", "stddev": "", "min": "1/1/2021", "max": "9/9/2022", "missing": "0", "distinct": "164"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "collection_week"}, {"transformationsData": [{"feature_label": "state", "transformation_label": "novalue"}], "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7422", "mean": "", "stddev": "", "min": "IL", "max": "IN", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "state"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "novalue"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7422", "mean": "", "stddev": "", "min": "ANACONDA", "max": "WOLF POINT", "missing": "0", "distinct": "51"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"feature_label": "hospital_subtype", "transformation_label": "novalue"}], "feature": "hospital_subtype", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7422", "mean": "", "stddev": "", "min": "Childrens Hospitals", "max": "Short Term", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hospital_subtype"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fips_code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "17195.54", "stddev": "305.23", "min": "17011", "max": "18091", "missing": "0"}, "updatedLabel": "fips_code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "is_metro_micro", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "0.6", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "is_metro_micro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_beds_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7422", "mean": "197.54", "stddev": "263.36", "min": "0.0", "max": "2190.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_beds_7_day_avg"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_used_covid_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7422", "mean": "5.49", "stddev": "17.52", "min": "0.0", "max": "216.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "inpatient_beds_used_covid..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "1360.14", "stddev": "1836.91", "min": "0", "max": "15330", "missing": "0"}, "updatedLabel": "total_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_adult_hospital_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "1187.24", "stddev": "1656.14", "min": "0", "max": "13510", "missing": "0"}, "updatedLabel": "all_adult_hospital_beds_7..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_adult_hospital_inpatient_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "1055.17", "stddev": "1516.45", "min": "0", "max": "12222", "missing": "0"}, "updatedLabel": "all_adult_hospital_inpati..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_used_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "94.21", "stddev": "193.95", "min": "0", "max": "2215", "missing": "0"}, "updatedLabel": "inpatient_beds_used_covid..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "86.32", "stddev": "186.19", "min": "0", "max": "2182", "missing": "0"}, "updatedLabel": "total_adult_patients_hosp..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_adult_patients_hospitalized_confirmed_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "26.26", "stddev": "81.76", "min": "0", "max": "1045", "missing": "0"}, "updatedLabel": "total_adult_patients_hosp..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "347.73", "stddev": "531.64", "min": "0", "max": "3479", "missing": "0"}, "updatedLabel": "inpatient_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_icu_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "74.86", "stddev": "190.37", "min": "0", "max": "1697", "missing": "0"}, "updatedLabel": "total_icu_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_staffed_adult_icu_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "48.9", "stddev": "133.9", "min": "0", "max": "945", "missing": "0"}, "updatedLabel": "total_staffed_adult_icu_b..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "icu_beds_used_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "136.58", "stddev": "299.34", "min": "0", "max": "2160", "missing": "0"}, "updatedLabel": "icu_beds_used_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "23.34", "stddev": "50.68", "min": "0", "max": "547", "missing": "0"}, "updatedLabel": "staffed_icu_adult_patient..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "staffed_icu_adult_patients_confirmed_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "18.61", "stddev": "42.81", "min": "0", "max": "510", "missing": "0"}, "updatedLabel": "staffed_icu_adult_patient..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous_day_covid_ED_visits_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "89.37", "stddev": "171.51", "min": "0", "max": "7473", "missing": "0"}, "updatedLabel": "previous_day_covid_ED_vis..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous_day_admission_adult_covid_suspected_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7422", "mean": "1.81", "stddev": "7.64", "min": "0", "max": "100", "missing": "0"}, "updatedLabel": "previous_day_admission_ad..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_pediatric_inpatient_bed_occupied_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7422", "mean": "4.89", "stddev": "32.12", "min": "0.0", "max": "312.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "all_pediatric_inpatient_b..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run delcolsHooks.ipynb
try:
	#sinkPreExecutionHook()

	coronahospcapaoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/666dc7d2c0455bf37f344f39/part-00000-7ca60322-b7fb-41c9-8474-ee15bf6e0899-c000.csv', 'samplecount': 5000, 'originalcount': 742253, 'orignalKey': '/FileStore/platform/testdata/1718387357966_filter_data_new.csv', 'pathOnly': '/TimeSeries/CoronaHospCapa', 'project_id': '666d962bc0455bf37f344af0', 'parent_id': '666d962bc0455bf37f344af0', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a8', 'field': 'collection_week', 'alias': 'collection_week', 'type': 'string', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a9', 'field': 'state', 'alias': 'state', 'type': 'string', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450aa', 'field': 'city', 'alias': 'city', 'type': 'string', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450ab', 'field': 'hospital_subtype', 'alias': 'hospital_subtype', 'type': 'string', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450ac', 'field': 'fips_code', 'alias': 'fips_code', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450ad', 'field': 'is_metro_micro', 'alias': 'is_metro_micro', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450ae', 'field': 'total_beds_7_day_avg', 'alias': 'total_beds_7_day_avg', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450af', 'field': 'inpatient_beds_used_covid_7_day_avg', 'alias': 'inpatient_beds_used_covid_7_day_avg', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b0', 'field': 'total_beds_7_day_sum', 'alias': 'total_beds_7_day_sum', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b1', 'field': 'all_adult_hospital_beds_7_day_sum', 'alias': 'all_adult_hospital_beds_7_day_sum', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b2', 'field': 'all_adult_hospital_inpatient_beds_7_day_sum', 'alias': 'all_adult_hospital_inpatient_beds_7_day_sum', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b3', 'field': 'inpatient_beds_used_covid_7_day_sum', 'alias': 'inpatient_beds_used_covid_7_day_sum', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b4', 'field': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b5', 'field': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b6', 'field': 'inpatient_beds_7_day_sum', 'alias': 'inpatient_beds_7_day_sum', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b7', 'field': 'total_icu_beds_7_day_sum', 'alias': 'total_icu_beds_7_day_sum', 'type': 'numeric', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b8', 'field': 'total_staffed_adult_icu_beds_7_day_sum', 'alias': 'total_staffed_adult_icu_beds_7_day_sum', 'type': 'numeric', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450b9', 'field': 'icu_beds_used_7_day_sum', 'alias': 'icu_beds_used_7_day_sum', 'type': 'numeric', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450ba', 'field': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'type': 'numeric', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450bb', 'field': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'type': 'numeric', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450bc', 'field': 'previous_day_covid_ED_visits_7_day_sum', 'alias': 'previous_day_covid_ED_visits_7_day_sum', 'type': 'numeric', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450bd', 'field': 'previous_day_admission_adult_covid_suspected_7_day_sum', 'alias': 'previous_day_admission_adult_covid_suspected_7_day_sum', 'type': 'numeric', 'position': '21'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450be', 'field': 'all_pediatric_inpatient_bed_occupied_7_day_avg', 'alias': 'all_pediatric_inpatient_bed_occupied_7_day_avg', 'type': 'real', 'position': '22'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345090', 'field': 'hospital_pk', 'alias': 'hospital_pk', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345091', 'field': 'collection_week', 'alias': 'collection_week', 'type': 'date', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345092', 'field': 'state', 'alias': 'state', 'type': 'String', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345093', 'field': 'city', 'alias': 'city', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345094', 'field': 'hospital_subtype', 'alias': 'hospital_subtype', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345095', 'field': 'fips_code', 'alias': 'fips_code', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345096', 'field': 'is_metro_micro', 'alias': 'is_metro_micro', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345097', 'field': 'total_beds_7_day_avg', 'alias': 'total_beds_7_day_avg', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345098', 'field': 'inpatient_beds_used_covid_7_day_avg', 'alias': 'inpatient_beds_used_covid_7_day_avg', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f345099', 'field': 'total_beds_7_day_sum', 'alias': 'total_beds_7_day_sum', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f34509a', 'field': 'all_adult_hospital_beds_7_day_sum', 'alias': 'all_adult_hospital_beds_7_day_sum', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f34509b', 'field': 'all_adult_hospital_inpatient_beds_7_day_sum', 'alias': 'all_adult_hospital_inpatient_beds_7_day_sum', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f34509c', 'field': 'inpatient_beds_used_covid_7_day_sum', 'alias': 'inpatient_beds_used_covid_7_day_sum', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f34509d', 'field': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f34509e', 'field': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f34509f', 'field': 'inpatient_beds_7_day_sum', 'alias': 'inpatient_beds_7_day_sum', 'type': 'numeric', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a0', 'field': 'total_icu_beds_7_day_sum', 'alias': 'total_icu_beds_7_day_sum', 'type': 'numeric', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a1', 'field': 'total_staffed_adult_icu_beds_7_day_sum', 'alias': 'total_staffed_adult_icu_beds_7_day_sum', 'type': 'numeric', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a2', 'field': 'icu_beds_used_7_day_sum', 'alias': 'icu_beds_used_7_day_sum', 'type': 'numeric', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a3', 'field': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'type': 'numeric', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a4', 'field': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'type': 'numeric', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a5', 'field': 'previous_day_covid_ED_visits_7_day_sum', 'alias': 'previous_day_covid_ED_visits_7_day_sum', 'type': 'numeric', 'position': '21'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a6', 'field': 'previous_day_admission_adult_covid_suspected_7_day_sum', 'alias': 'previous_day_admission_adult_covid_suspected_7_day_sum', 'type': 'numeric', 'position': '22'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666dca68c0455bf37f3450a7', 'field': 'all_pediatric_inpatient_bed_occupied_7_day_avg', 'alias': 'all_pediatric_inpatient_bed_occupied_7_day_avg', 'type': 'real', 'position': '23'}], 'server': 'https://dnm.bfirst.ai:443', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'dataset.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-ad328e37-4b04-479a-9be6-b2b2d8799620-c000.csv', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
